## Simplex Method.
# By Mauricio Soroco

2022-09-16

In this portfolio object, we will be using the pivot function (taken from Patrick's notes below) to implement the two phase simplex method. I've included Patrick's pivot operation before my work below.

In [2]:
import numpy as np

## Pivot Operation (taken from Patrick's notes)

Assume for now that $\mathbf{b} \geq 0$. Construct the initial **tableau** matrix:

$$
T = 
\begin{bmatrix}
A & I & \mathbf{b} \\
\mathbf{c}^T & 0 & 0
\end{bmatrix}
$$

My definition is different from other sources. I put $\mathbf{c}^T$ in the bottom row because I want the row indices of $T$ to match the indices of the slack variables. Like row 0 of $T$ corresponds to $w_0$, row 1 corresponds to $w_1$, etc.

The **basic variables** correspond to the columns of $T$ which are columns of the identity matrix (in the first $m$ rows). In the initial tableau, the slack variables $w_0,\dots,w_{m-1}$ are the basic variables and the decision variables $x_0,\dots,x_{n-1}$ are the nonbasic variables. The corresponding vertex is $\mathbf{x}=0$.

We want to move to an adjacent vertex by selecting a new basic variable (the **entering variable**) and removing an existing basic variable (the **leaving variable**).

We choose the entering and leaving variables such that:

* The value of the objective function increases.
* The new list of basic variables defines a vertex. In other words, the values of the basic variables (with nonbasic variables set to 0) are all positive.

How do we do this? Choose the entering variable such that it has a positive coefficient in the objective function. Increasing the value of this variable will increase the value of the objective function.

Choose the leaving variable such that the values of the basic variables (with nonbasic variables set to 0) are all positive. To do this, let $k$ be the column index of the entering variable. Look at the ratios

$$
\frac{t_{i,n+m}}{t_{i,k}} \ , \ i = 0,\dots,m-1
$$

where $t_{i,n+m}$ are the entries in the last column of $T$, and $t_{i,k}$ are the entries in the $k$th column of $T$. Select row $i$ such that this ratio is postive and minimal.

Let $k$ be the column index of the entering variable. Let $\ell$ be the row index of the leaving variable. The pivot operation applies row operations to the tableau $T$ such that column $k$ has 1 in row $\ell$ and all other entries 0. The last column of the tableau should always have positive entries in rows $0,\dots,m-1$.

Below is a function called `pivot` which takes a tableau matrix $T$ and indices `k` and `l`. The function performs the pivot operation on $T$ and returns the new tableau matrix.

In [3]:
def pivot(T,k,l):
    E = np.eye(T.shape[0])
    E[:,l] = -T[:,k]/T[l,k]
    E[l,l] = 1/T[l,k]
    return E@T

## Simplex Method.
### By Mauricio Soroco 60785250

In the code below, we make use of the pivot operation above to implement the two phase simplex method. The Simplex Method below is implemented with Anstees Rule:
- Choose the entering variable with the largest positive coefficient.
- If there is a tie, then choose the one with the smaller subscript.
- If there is a choice of leaving variable, then choose the one with the smallest subscript.

We apply these to three examples:
- Problem with an optimal solution needing only Phase II of the simplex method
- Problem with an optimal solution needing the two phase simplex method
- Problem with an optimal solution needing the two phase simplex method
- Problem with no feasible solution needing the two phase simplex method (particularly the first phase to show the problem is infeasible).

## Code
We first define some helpful helper methods for our implementation

In [4]:
""" Prints the solution found by the simplex method. Includes the final tableau,
    the value of the objective function, and the values for the original decision variables.
    Params: Tableau T
"""
def report_solution(T):
    T = T.round(2)
    print("Final tableau:")
    print(T)
    (r, c) = T.shape
    z = T[r-1,:]
    print("\nThe optimal objective value is: " + str(0-z[c-1]) + "\n")
    num_nonbasic = c - (r - 1) - 1
    for j in range(num_nonbasic):
        indices = np.nonzero(T[:, j])
        nonzeros = T[:, j][indices]
        if (nonzeros.size == 1 and nonzeros[0] == 1):
            print("x_" + str(j) +" = " + str(T[indices[0][0], c-1]))
        else:
            print("x_" + str(j) +" = 0")

In [5]:
""" Computes the new objective function, given the current set of
    basic and non-basic variables (encoded in the matrix A).
    
    This method gets called after Phase I of the simplex method
    ends and we need to resubstitute original objective function
    into the tableau, but in terms of the current set of basic
    and non-basic variables.

    Params: A           The matrix A from the tableau
            original_c  The original coefficients of the objective function
                        to rearrange.
"""
def compute_new_c(A, original_c):
    (r, c) = A.shape
    new_c = original_c.copy()
    for j in range(c):
        indices = np.nonzero(A[:, j])
        nonzeros = A[:, j][indices]
        if (nonzeros.size == 1 and nonzeros[0] == 1):
            basic_var = j
            row_of_basic_var = indices[0][0]
            for k in range(c-1):
                new_c[k] = new_c[k] - original_c[j] * A[row_of_basic_var, k]
            new_c[c-1] = new_c[c-1] + original_c[j] * A[row_of_basic_var, c-1] # the last column of c
    new_c[c-1] = 0-new_c[c-1]
    return new_c

In [6]:
""" Finds the smallest basic variable index out of two basic variables.
    Params: Tableau T
            Basic Variable candidates a & b """
def smallest_basic(T, a, b):
    (r, c) = T.shape
    for j in range(c):
        indices = np.nonzero(T[:, j])
        nonzeros = T[:, j][indices]
        if (nonzeros.size == 1 and nonzeros[0] == 1):
            if ((indices[0] == a) or (indices[0] == b)):
                return indices[0][0]
    raise Exception("Something went wrong")


## Phase II
The code below performs phase II of the simplex method on the given tableau. It is assumed that Phase I was already performed if necessary.

In [7]:
# Anstees Rule
def PhaseII(T):
    (m, k) = T.shape
    c = T[m-1,:]

    while(any((c[0:k-1] > 0))):
        # choose entering variable with largest positive coefficient (if tie, pick smallest subscript)
        entering_col = -1
        largest = 0
        for i in range(k - 1):
            if c[i] > largest:
                entering_col = i
                largest = c[i]
        if entering_col == -1:
            print("All coefficients are negative.")
            raise Exception("Entering column invalid")
        # choose leaving variable with smallest ratio
        leaving_row = -1
        min_ratio = np.Infinity
        for i in range(m - 1):
            ratio = T[i, k-1] / T[i, entering_col]
            if((ratio == min_ratio) and (ratio >= 0)):
                #tie(leaving_row, i), find the column with the smallest subscript.
                leaving_row = smallest_basic(T, leaving_row, i)
            if((ratio < min_ratio) and (ratio >= 0)): 
                # in accordance with anstees rule we use smallest subscript for ratio ties
                min_ratio = ratio
                leaving_row = i
        if leaving_row == -1:
            raise Exception("Leaving row invalid")
        # now that we have entering and leaving variables we can pivot
        print("\nentering: col " + str(entering_col) + "     Leaving: row "+ str(leaving_row) + "\n")
        T = pivot(T, entering_col, leaving_row)
        print(T)
        # update c:
        c = T[m-1,:]
        # when phaseII is finished:
    print("\nSince all the coefficients of the objective function are negative we reached the optimal solution.\n")
    # print(T)
    return T



## Phase I
The code below performs phase I of the simplex method on the given tableau

In [8]:
def PhaseI(T):
    (m, k) = T.shape
    b = T[:,k-1]
    original_c = T[m-1,:]
    ideal_c = []
    if (any(b < 0)):
        print("\nAuxiliary Problem\n") # Auxiliary problem required
        A = T[0:m-1 ,0:k - (m -1)-1]
        N = T[:, k - (m -1)-1:k]
        T = np.hstack([np.repeat(-1, m).reshape(m,1), np.vstack([A, np.zeros(k - (m -1)-1)]), N])
        ideal_c = T[m-1,:]
        print(T)
        # now we do initial pivot:
        entering_col = 0
        # choose leaving variable with most negative b
        leaving_row = -1
        min_value = 0
        for i in range(m - 1):
            if(b[i] < min_value):
                min_value = b[i]
                leaving_row = i
        if leaving_row == -1:
            raise Exception("Leaving row invalid")
        print("\nentering: col " + str(entering_col) + "     Leaving: row "+ str(leaving_row) + "\n")
        T = pivot(T, entering_col, leaving_row)
        print(T)
        try:
            T = PhaseII(T)
            T = T.round(8)
            # check for feasibility and resubstitute c
            if all(ideal_c == T[m-1, :]):
                A = T[0:m-1 ,1:k+1]
                
                print("\nAuxiliary Problem optimal with x_0 = 0. Substitute initial obj. function\n")# compute new c:
                new_c = compute_new_c(A, original_c)
                T = np.vstack([A, new_c])
                print(T) # begin simplex method on this matrix.
            else:
                print(T)
                raise Exception("\nAuxiliary problem does not have optimal value y = 0. Original problem is infeasible\n")
        except(BaseException):
            print("\nAuxiliary problem does not have optimal value y = 0. Original problem is infeasible\n")
            raise Exception("\nAuxiliary problem does not have optimal value y = 0. Original problem is infeasible\n")
    return T
        



## Two-Phase Simplex Method
This is a wrapper function for combining the two phases of the simplex method above to produce the two-phase simplex method

In [9]:
def Simplex(T):
    print("Initial Tableau:\n")
    print(T)
    T = PhaseI(T)
    T = PhaseII(T)
    report_solution(T)
    # return T

# Examples

## Example 1.
A non-degenerate problem with optimal solution, therefore only needs phase II of the simplex method.
Taken from the Vanderbei textbook, Exercise 2.1

$$
A = \begin{bmatrix} 2 & 1 & 1 & 3 \\ 1 & 3 & 1 & 2 \end{bmatrix}
\hspace{1in}
\mathbf{b} = \begin{bmatrix} 5 \\ 3 \end{bmatrix}
\hspace{1in}
\mathbf{c} = \begin{bmatrix} 6 \\ 8 \\ 5 \\ 9 \end{bmatrix}
$$

In [10]:
# non-degenerate problem with optimal solution. 
A = np.array([[2.,1.,1.,3.],[1.,3.,1.,2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([6.,8.,5.,9.])
b = np.array([5.,3.])
T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[2. 1. 1. 3. 1. 0. 5.]
 [1. 3. 1. 2. 0. 1. 3.]
 [6. 8. 5. 9. 0. 0. 0.]]


In [11]:
Simplex(T)

Initial Tableau:

[[2. 1. 1. 3. 1. 0. 5.]
 [1. 3. 1. 2. 0. 1. 3.]
 [6. 8. 5. 9. 0. 0. 0.]]

entering: col 3     Leaving: row 1

[[  0.5  -3.5  -0.5   0.    1.   -1.5   0.5]
 [  0.5   1.5   0.5   1.    0.    0.5   1.5]
 [  1.5  -5.5   0.5   0.    0.   -4.5 -13.5]]

entering: col 0     Leaving: row 0

[[  1.  -7.  -1.   0.   2.  -3.   1.]
 [  0.   5.   1.   1.  -1.   2.   1.]
 [  0.   5.   2.   0.  -3.   0. -15.]]

entering: col 1     Leaving: row 1

[[ 1.0000000e+00 -4.4408921e-16  4.0000000e-01  1.4000000e+00
   6.0000000e-01 -2.0000000e-01  2.4000000e+00]
 [ 0.0000000e+00  1.0000000e+00  2.0000000e-01  2.0000000e-01
  -2.0000000e-01  4.0000000e-01  2.0000000e-01]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00 -1.0000000e+00
  -2.0000000e+00 -2.0000000e+00 -1.6000000e+01]]

entering: col 2     Leaving: row 1

[[ 1.00000000e+00 -2.00000000e+00 -2.22044605e-17  1.00000000e+00
   1.00000000e+00 -1.00000000e+00  2.00000000e+00]
 [ 0.00000000e+00  5.00000000e+00  1.00000000e+00  1.00000000e

## Example 2.
An auxiliary problem with an optimal auxiliary solution (for phase I) and a feasible original solution (for phase II).
Taken from the Vanderbei textbook (exercise 2.3)

In [12]:
# #Auxiliary problem with an optimal Aux solution and feasible original solution
# Vanderbei (exercise 2.3)
A = np.array([[-1.,-1.,-1.],[2.,-1.,1.]])
m,n = A.shape
I = np.eye(m)
c = np.array([2.,-6.,0.])
b = np.array([-2.,1.])
T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

Simplex(T)

[[-1. -1. -1.  1.  0. -2.]
 [ 2. -1.  1.  0.  1.  1.]
 [ 2. -6.  0.  0.  0.  0.]]
Initial Tableau:

[[-1. -1. -1.  1.  0. -2.]
 [ 2. -1.  1.  0.  1.  1.]
 [ 2. -6.  0.  0.  0.  0.]]

Auxiliary Problem

[[-1. -1. -1. -1.  1.  0. -2.]
 [-1.  2. -1.  1.  0.  1.  1.]
 [-1.  0.  0.  0.  0.  0.  0.]]

entering: col 0     Leaving: row 0

[[ 1.  1.  1.  1. -1.  0.  2.]
 [ 0.  3.  0.  2. -1.  1.  3.]
 [ 0.  1.  1.  1. -1.  0.  2.]]

entering: col 1     Leaving: row 1

[[ 1.00000000e+00  5.55111512e-17  1.00000000e+00  3.33333333e-01
  -6.66666667e-01 -3.33333333e-01  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  6.66666667e-01
  -3.33333333e-01  3.33333333e-01  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.33333333e-01
  -6.66666667e-01 -3.33333333e-01  1.00000000e+00]]

entering: col 2     Leaving: row 0

[[ 1.00000000e+00  5.55111512e-17  1.00000000e+00  3.33333333e-01
  -6.66666667e-01 -3.33333333e-01  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e

C:\Users\mauri\AppData\Local\Temp/ipykernel_10376/275364688.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = T[i, k-1] / T[i, entering_col]


## Example 3
An auxiliary problem with no optimal auxiliary solution (thus original problem is infeasible).

In [13]:
#Auxiliary problem with no optimal Aux solution
A = np.array([[1., 1.],[2.,-2.],[-3, -2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([3.,1.])
b = np.array([1.,-1., -4.])
T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[ 1.  1.  1.  0.  0.  1.]
 [ 2. -2.  0.  1.  0. -1.]
 [-3. -2.  0.  0.  1. -4.]
 [ 3.  1.  0.  0.  0.  0.]]


In [14]:
Simplex(T)

Initial Tableau:

[[ 1.  1.  1.  0.  0.  1.]
 [ 2. -2.  0.  1.  0. -1.]
 [-3. -2.  0.  0.  1. -4.]
 [ 3.  1.  0.  0.  0.  0.]]

Auxiliary Problem

[[-1.  1.  1.  1.  0.  0.  1.]
 [-1.  2. -2.  0.  1.  0. -1.]
 [-1. -3. -2.  0.  0.  1. -4.]
 [-1.  0.  0.  0.  0.  0.  0.]]

entering: col 0     Leaving: row 2

[[ 0.  4.  3.  1.  0. -1.  5.]
 [ 0.  5.  0.  0.  1. -1.  3.]
 [ 1.  3.  2.  0.  0. -1.  4.]
 [ 0.  3.  2.  0.  0. -1.  4.]]

entering: col 1     Leaving: row 1

[[ 0.00000000e+00 -2.22044605e-16  3.00000000e+00  1.00000000e+00
  -8.00000000e-01 -2.00000000e-01  2.60000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   2.00000000e-01 -2.00000000e-01  6.00000000e-01]
 [ 1.00000000e+00  0.00000000e+00  2.00000000e+00  0.00000000e+00
  -6.00000000e-01 -4.00000000e-01  2.20000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e+00  0.00000000e+00
  -6.00000000e-01 -4.00000000e-01  2.20000000e+00]]

entering: col 2     Leaving: row 0

[[ 0.00000000e+00 -7.40

C:\Users\mauri\AppData\Local\Temp/ipykernel_10376/275364688.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = T[i, k-1] / T[i, entering_col]


Exception: 
Auxiliary problem does not have optimal value y = 0. Original problem is infeasible


## Example 4
An auxiliary problem with an optimal auxiliary solution. Original problem has an optimal solution.

In [ ]:
#Auxiliary problem with an optimal Aux solution
A = np.array([[2., 1., 1],[-3.,-4., -2.]])
m,n = A.shape
I = np.eye(m)
c = np.array([3.,2., 3.])
b = np.array([2.,-8.])
T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[ 2.  1.  1.  1.  0.  2.]
 [-3. -4. -2.  0.  1. -8.]
 [ 3.  2.  3.  0.  0.  0.]]


In [ ]:
Simplex(T)

Initial Tableau:

[[ 2.  1.  1.  1.  0.  2.]
 [-3. -4. -2.  0.  1. -8.]
 [ 3.  2.  3.  0.  0.  0.]]

Auxiliary Problem

[[-1.  2.  1.  1.  1.  0.  2.]
 [-1. -3. -4. -2.  0.  1. -8.]
 [-1.  0.  0.  0.  0.  0.  0.]]

entering: col 0     Leaving: row 1

[[ 0.  5.  5.  3.  1. -1. 10.]
 [ 1.  3.  4.  2.  0. -1.  8.]
 [ 0.  3.  4.  2.  0. -1.  8.]]

entering: col 2     Leaving: row 1

[[-1.25  1.25  0.    0.5   1.    0.25  0.  ]
 [ 0.25  0.75  1.    0.5   0.   -0.25  2.  ]
 [-1.    0.    0.    0.    0.    0.    0.  ]]
Since all the coefficients of the objective function are negative we reached the optimal solution.


Auxiliary Problem optimal with x_0 = 0. Substitute initial obj. function

[[ 1.25  0.    0.5   1.    0.25  0.  ]
 [ 0.75  1.    0.5   0.   -0.25  2.  ]
 [ 1.5   0.    2.    0.    0.5  -4.  ]]

entering: col 2     Leaving: row 0

[[ 2.5  0.   1.   2.   0.5  0. ]
 [-0.5  1.   0.  -1.  -0.5  2. ]
 [-3.5  0.   0.  -4.  -0.5 -4. ]]
Since all the coefficients of the objective function